# PROJECT 3

From the existing **companies**, choose one to steal their current venue 🥷: query and filter the database based on some of your criteria. Then use an API to do queries (from those companies) and check the companies' surroundings to check the other criteria.

Your result will be `coordinates`.

## Importing stuff and Looking at the companies json file

In [1]:
import pandas as pd 

import geopandas as gdp
from cartoframes.viz import Map, Layer, popup_element

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

import requests

from pymongo import MongoClient
from pymongo import GEOSPHERE
import json

client = MongoClient("localhost:27017")
client.list_database_names()

['Ironhack', 'admin', 'bookstores', 'config', 'local']

### Setting DB and collection 

In [2]:
def setting_db_collection():
    db = client["Ironhack"]
    c = db.get_collection("companies")

## Exploring different categories

In [3]:
print(c.distinct("category_code"))

[None, 'advertising', 'analytics', 'automotive', 'biotech', 'cleantech', 'consulting', 'design', 'ecommerce', 'education', 'enterprise', 'fashion', 'finance', 'games_video', 'government', 'hardware', 'health', 'hospitality', 'legal', 'local', 'manufacturing', 'medical', 'messaging', 'mobile', 'music', 'nanotech', 'network_hosting', 'news', 'nonprofit', 'other', 'photo_video', 'public_relations', 'real_estate', 'search', 'security', 'semiconductor', 'social', 'software', 'sports', 'transportation', 'travel', 'web']


 - We can see that there are design and games_video companies. Lets see how many of each do we have in our companies collection

In [4]:
print("Number of gaming companies: ",len(list(c.find({"category_code": "games_video"}))))
print("---------------------------------------------------------------------------------")
print("Number of design companies: ",len(list(c.find({"category_code": "design"}))))

Number of gaming companies:  1083
---------------------------------------------------------------------------------
Number of design companies:  4


## Checking design companies

In [5]:
filter1 = {"category_code": "design"}
projection1 = {"_id": 0, "name": 1,
               "offices.country_code":1,
               "offices.latitude":1,
               "offices.longitude":1 }

design_list = list(c.find(filter1 , projection1).sort("number_of_employees",-1).limit(10))
design_list

[{'name': '99designs',
  'offices': [{'country_code': 'USA',
    'latitude': 37.7955307,
    'longitude': -122.4005983},
   {'country_code': 'AUS', 'latitude': -37.8026587, 'longitude': 144.9868546},
   {'country_code': 'DEU', 'latitude': 52.4986199, 'longitude': 13.4469031}]},
 {'name': 'Graticle',
  'offices': [{'country_code': 'USA',
    'latitude': 45.7968414,
    'longitude': -122.6931439}]},
 {'name': 'Kickstarter',
  'offices': [{'country_code': 'USA', 'latitude': None, 'longitude': None}]},
 {'name': 'Moonfruit',
  'offices': [{'country_code': 'GBR', 'latitude': None, 'longitude': None}]}]

- Filtering the ones from the US

In [6]:
filter1 = {"category_code": "design","offices.country_code":"USA"}
projection1 = {"_id": 0, "name": 1,
               "offices.country_code":1,
               "offices.latitude":1,
               "offices.longitude":1 }

design_list_USA = list(c.find(filter1 , projection1).sort("number_of_employees",-1).limit(10))
design_list_USA

[{'name': '99designs',
  'offices': [{'country_code': 'USA',
    'latitude': 37.7955307,
    'longitude': -122.4005983},
   {'country_code': 'AUS', 'latitude': -37.8026587, 'longitude': 144.9868546},
   {'country_code': 'DEU', 'latitude': 52.4986199, 'longitude': 13.4469031}]},
 {'name': 'Graticle',
  'offices': [{'country_code': 'USA',
    'latitude': 45.7968414,
    'longitude': -122.6931439}]},
 {'name': 'Kickstarter',
  'offices': [{'country_code': 'USA', 'latitude': None, 'longitude': None}]}]

### Converting the data into a readable df

In [7]:
df_design = pd.DataFrame(design_list_USA)
df_design = df_design.explode('offices')
df_design = df_design.merge(df_design['offices'].apply(pd.Series), left_index=True, right_index=True)
df_design = df_design[['name', 'country_code','latitude', 'longitude']]
df_design

,name,country_code,latitude,longitude
0,99designs,USA,37.795531,-122.400598
0,99designs,AUS,-37.802659,144.986855
0,99designs,DEU,52.498620,13.446903
0,99designs,USA,37.795531,-122.400598
0,99designs,AUS,-37.802659,144.986855
0,99designs,DEU,52.498620,13.446903
0,99designs,USA,37.795531,-122.400598
0,99designs,AUS,-37.802659,144.986855
0,99designs,DEU,52.498620,13.446903
1,Graticle,USA,45.796841,-122.693144


- It looks like we have a company with venues in several countries, so let's focus on the ones from the USA  

In [8]:
df_design_USA = df_design[df_design["country_code"] == "USA"]
df_design_USA

,name,country_code,latitude,longitude
0,99designs,USA,37.795531,-122.400598
0,99designs,USA,37.795531,-122.400598
0,99designs,USA,37.795531,-122.400598
1,Graticle,USA,45.796841,-122.693144
2,Kickstarter,USA,NaN,NaN


- Removing duplicates and nan elements (kickstarter doesn't seem to have a venue)

In [9]:
df_design_USA.drop_duplicates(inplace=True)
df_design_USA.dropna(inplace=True)
df_design_USA

C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\1160257017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_design_USA.drop_duplicates(inplace=True)
C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\1160257017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_design_USA.dropna(inplace=True)


,name,country_code,latitude,longitude
0,99designs,USA,37.795531,-122.400598
1,Graticle,USA,45.796841,-122.693144


### First map to explore the final two venues

In [10]:
us_lat = 37.795531
us_lon = -122.400598

us_map = Map(location=[us_lat, us_lon], zoom_start = 5)

# Convert the dataframe to a GeoDataFrame
gdf = gdp.GeoDataFrame(df_design_USA, geometry=gdp.points_from_xy(df_design_USA.longitude, df_design_USA.latitude))
# Create a folium map centered in Barcelona
us_map = folium.Map(location=[us_lat, us_lon], zoom_start=4.4)
# Loop through each row of the GeoDataFrame and add a marker for each coffee shop
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.latitude, row.longitude],
                  popup=row['name']).add_to(us_map)
# Display the map
us_map

- After a quick look on the two design companies we have decided to focus on '99designs' (Not much going on the Vancouver venue)

In [11]:
c.find_one({"name": "99designs"}, {"_id":0, "name:":1, "category_code": 1, "number_of_employees":1,"tag_list":1 })

{'category_code': 'design',
 'number_of_employees': 100,
 'tag_list': 'graphic-design, crowdsourcing, online-marketplace, marketplace, crowdsourced-design'}

## Check nearby companies

In [12]:
# Saving this one: "total_money_raised":{"$gte":1000000}

filter1 = {"offices.country_code":"USA", "offices.state_code": "CA"}

projection1 = {"_id": 0, "name": 1,
               "offices.state_code":1,
               "total_money_raised":1,
               "offices.latitude":1,
               "offices.longitude":1}

CA_companies = list(c.find(filter1 , projection1).sort("number_of_employees",-1))

### Cleaning and transforming the data

In [13]:
# Transforming the list of dictionaries into a df

df_1 = pd.DataFrame(CA_companies)
df_1 = df_1.explode('offices')
df_1 = df_1.reset_index(drop=True)
df_2 = pd.DataFrame(df_1['offices'].tolist())
df_merged = pd.merge(df_1, df_2, left_index=True, right_index=True)

df = df_merged[df_merged["state_code"] == "CA"]
df.drop(columns=["offices"], inplace=True)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True ,inplace=True)
df


C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\2326541423.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["offices"], inplace=True)
C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\2326541423.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\2326541423.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_d

,name,total_money_raised,state_code,latitude,longitude
0,PayPal,$197M,CA,37.294465,-121.927696
1,Sony,$0,CA,37.090240,-95.712891
2,Intel,$0,CA,37.384949,-121.967610
3,Apple,$0,CA,37.330534,-122.028961
4,Cisco,$2.5M,CA,37.408802,-121.953770
...,...,...,...,...,...
2827,Cantimer,$2M,CA,37.484616,-122.181753
2828,Celestial Semiconductor,$2M,CA,37.386417,-121.929170
2829,Purfresh,$62.7M,CA,37.477008,-122.151599
2830,InSound Medical,$11M,CA,37.512038,-121.998513


### Creating a map with the filtered companies

In [14]:

ca_lat = 37.795531
ca_lon = -122.400598

ca_map = Map(location=[ca_lat, ca_lon], zoom_start = 5)
#us_map


# Convert the dataframe to a GeoDataFrame
ca_1 = gdp.GeoDataFrame(df, geometry=gdp.points_from_xy(df.longitude, df.latitude))

# Create a folium map centered in California
ca_map = folium.Map(location=[ca_lat, ca_lon], zoom_start=4.75)

# Loop through each row of the GeoDataFrame and add a marker for each coffee shop
for idx, row in ca_1.iterrows():
    folium.Marker(location=[row.latitude, row.longitude],
                  popup=row['name']).add_to(ca_map)
# Display the map
ca_map

### Filtering by distance

In [15]:
my_location = [ 37.795531, -122.400598]

def type_point (list_):
    formatted_dict_ = {"type": "Point", "coordinates": list_}
    return formatted_dict_

max_distance = 500

In [16]:
converted_location = type_point(my_location)

In [17]:

import math
def haversine(coord1, coord2):
 
    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    km = round(km, 3)

    return meters


In [18]:
df['Distance_meters'] = df.apply(lambda x: haversine ([x['longitude'],x['latitude']],[-122.400598, 37.795531]),axis=1)
df.head(5)

C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\1329909278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Distance_meters'] = df.apply(lambda x: haversine ([x['longitude'],x['latitude']],[-122.400598, 37.795531]),axis=1)


,name,total_money_raised,state_code,latitude,longitude,geometry,Distance_meters
0,PayPal,$197M,CA,37.294465,-121.927696,POINT (-121.92770 37.29447),69588.259
1,Sony,$0,CA,37.090240,-95.712891,POINT (-95.71289 37.09024),2349400.146
2,Intel,$0,CA,37.384949,-121.967610,POINT (-121.96761 37.38495),59496.192
3,Apple,$0,CA,37.330534,-122.028961,POINT (-122.02896 37.33053),61208.163
4,Cisco,$2.5M,CA,37.408802,-121.953770,POINT (-121.95377 37.40880),58298.243


In [19]:
# Filtering close companies
close_companies = df[df['Distance_meters'] <= 500]
close_companies.sort_values(by=['Distance_meters'],inplace=True)
close_companies

C:\Users\DavidVicente\AppData\Local\Temp\ipykernel_20488\2892593828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_companies.sort_values(by=['Distance_meters'],inplace=True)


,name,total_money_raised,state_code,latitude,longitude,geometry,Distance_meters
281,99designs,$35M,CA,37.795531,-122.400598,POINT (-122.40060 37.79553),0.043
363,Weebly,$670k,CA,37.795555,-122.400729,POINT (-122.40073 37.79556),11.816
444,Visage Mobile,$14M,CA,37.795141,-122.401194,POINT (-122.40119 37.79514),67.993
1622,BrightRoll,$46M,CA,37.794643,-122.401144,POINT (-122.40114 37.79464),109.779
2312,LiveVox,$12.1M,CA,37.794643,-122.401144,POINT (-122.40114 37.79464),109.779
...,...,...,...,...,...,...,...
1926,Zinio,$27M,CA,37.791500,-122.400450,POINT (-122.40045 37.79150),448.415
2104,GoPlanit,$500k,CA,37.791576,-122.399363,POINT (-122.39936 37.79158),452.967
1617,Gydget,$1M,CA,37.791733,-122.398398,POINT (-122.39840 37.79173),464.459
376,Bizo,$28.5M,CA,37.791119,-122.401171,POINT (-122.40117 37.79112),493.210


### Map of the companies that are less than 500 meters away

In [20]:
sa_lat = 37.795531
sa_lon = -122.400598

sa_map = Map(location=[sa_lat, sa_lon], zoom_start = 5)
#us_map


# Convert the dataframe to a GeoDataFrame
sa_1 = gdp.GeoDataFrame(close_companies, geometry=gdp.points_from_xy(close_companies.longitude, close_companies.latitude))

# Create a folium map centered in California
sa_map = folium.Map(location=[sa_lat, sa_lon], zoom_start=17)

# Loop through each row of the GeoDataFrame and add a marker for each coffee shop
for idx, row in sa_1.iterrows():
    folium.Marker(location=[row.latitude, row.longitude],
                  popup=row['name']).add_to(sa_map)
# Display the map
sa_map

## Looking for starbucks, pre-school, vegan friendly restaurants & bars

In [21]:
from getpass import getpass
token = getpass()

 ················································


### Getting nearby starbucks

In [22]:
url = "https://api.foursquare.com/v3/places/search?query=starbucks&ll=37.795531%2C-122.400598&sort=DISTANCE&limit=5"

headers = {
    "accept": "application/json",
    "Authorization": f"{token}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4a9809d8f964a5202e2a20e3","categories":[{"id":13035,"name":"Coffee Shop","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_","suffix":".png"}}],"chains":[{"id":"ab4c54c0-d68a-012e-5619-003048cad9da","name":"Starbucks"}],"distance":48,"geocodes":{"main":{"latitude":37.795231,"longitude":-122.400412},"roof":{"latitude":37.795231,"longitude":-122.400412}},"link":"/v3/places/4a9809d8f964a5202e2a20e3","location":{"address":"425 Battery St","address_extended":"Ste A","census_block":"060750611012021","country":"US","cross_street":"at Clay St","dma":"San Francisco-Oakland-San Jose","formatted_address":"425 Battery St (at Clay St), San Francisco, CA 94111","locality":"San Francisco","postcode":"94111","region":"CA"},"name":"Starbucks","related_places":{"parent":{"fsq_id":"5383ee97498e0086eaf1acaa","name":"Club Quarters"}},"timezone":"America/Los_Angeles"},{"fsq_id":"49d0da91f964a520395b1fe3","categories":[{"id":13035,"name":"Coffee Shop","icon":{"pr

In [23]:
coordinates = response.json()["results"][0]["geocodes"]["main"]
coordinates

{'latitude': 37.795231, 'longitude': -122.400412}

In [24]:
lat, lon = coordinates["latitude"], coordinates["longitude"]

In [25]:
name = response.json()["results"][0]["name"]
name

'Starbucks'

In [26]:
def name_coordinates (dict_):
    
    processed_dict = {"name": dict_["name"],
                     "lat": dict_["geocodes"]["main"]["latitude"],
                     "lon": dict_["geocodes"]["main"]["longitude"]}
    
    return processed_dict

In [27]:
new_list = []
for i in response.json()["results"]:
    new_list.append(name_coordinates(i))
new_list

[{'name': 'Starbucks', 'lat': 37.795231, 'lon': -122.400412},
 {'name': 'Starbucks', 'lat': 37.794601, 'lon': -122.402669},
 {'name': 'Starbucks', 'lat': 37.793025, 'lon': -122.39978},
 {'name': 'Starbucks', 'lat': 37.797015, 'lon': -122.398007},
 {'name': 'Starbucks', 'lat': 37.793818, 'lon': -122.396678}]

In [28]:
close_5_starbucks = pd.DataFrame(new_list)

### Getting nearby pre-schools

In [29]:
url = "https://api.foursquare.com/v3/places/search?query=Preschool&ll=37.795531%2C-122.400598&sort=DISTANCE&limit=3"

headers = {
    "accept": "application/json",
    "Authorization": f"{token}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"558a380a498e13330910aef6","categories":[{"id":12058,"name":"Elementary School","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/school_","suffix":".png"}}],"chains":[],"distance":367,"geocodes":{"main":{"latitude":37.795777,"longitude":-122.404818},"roof":{"latitude":37.795777,"longitude":-122.404818}},"link":"/v3/places/558a380a498e13330910aef6","location":{"address":"838 Kearny St","census_block":"060750611012004","country":"US","cross_street":"btwn Washington & Jackson St","dma":"San Francisco-Oakland-San Jose","formatted_address":"838 Kearny St (btwn Washington & Jackson St), San Francisco, CA 94108","locality":"San Francisco","postcode":"94108","region":"CA"},"name":"Saint Marys Chinese Day School","related_places":{},"timezone":"America/Los_Angeles"},{"fsq_id":"5cc0a9cac0c0840025deceea","categories":[{"id":12056,"name":"Preschool","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/school_","suffix":".png"}}],"chains":[],"dist

In [30]:
kinder_list = []
for i in response.json()["results"]:
    kinder_list.append(name_coordinates(i))
kinder_list

[{'name': 'Saint Marys Chinese Day School',
  'lat': 37.795777,
  'lon': -122.404818},
 {'name': 'Seabird Preschool', 'lat': 37.797109, 'lon': -122.407907},
 {'name': 'Infant Care State Preschool Center',
  'lat': 37.796529,
  'lon': -122.408232}]

In [31]:
close_kinder = pd.DataFrame(kinder_list)


### Getting nearby pubs

In [32]:
url = "https://api.foursquare.com/v3/places/search?query=nightclub&ll=37.795531%2C-122.400598&sort=DISTANCE&limit=3"

headers = {
    "accept": "application/json",
    "Authorization": f"{token}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4f32bd4d19836c91c7f53884","categories":[{"id":10032,"name":"Night Club","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_","suffix":".png"}}],"chains":[],"distance":203,"geocodes":{"main":{"latitude":37.79575,"longitude":-122.403024},"roof":{"latitude":37.79575,"longitude":-122.403024}},"link":"/v3/places/4f32bd4d19836c91c7f53884","location":{"address":"582 Washington St","census_block":"060750611012007","country":"US","dma":"San Francisco-Oakland-San Jose","formatted_address":"582 Washington St, San Francisco, CA 94111","locality":"San Francisco","postcode":"94111","region":"CA"},"name":"Avantine","related_places":{},"timezone":"America/Los_Angeles"},{"fsq_id":"42829c80f964a5201f221fe3","categories":[{"id":13016,"name":"Lounge","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightlife/default_","suffix":".png"}},{"id":13068,"name":"American Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","s

In [33]:
club_list = []
for i in response.json()["results"]:
    club_list.append(name_coordinates(i))
club_list

[{'name': 'Avantine', 'lat': 37.79575, 'lon': -122.403024},
 {'name': 'Kells Irish Restaurant & Pub',
  'lat': 37.796482,
  'lon': -122.404142},
 {'name': 'Hue SF', 'lat': 37.797934, 'lon': -122.404801}]

In [34]:
pub = pd.DataFrame(club_list)


### Getting nearby vegan-friendly restaurants

In [35]:
url = "https://api.foursquare.com/v3/places/search?query=VeganRestaurant&ll=37.795531%2C-122.400598&sort=DISTANCE&limit=3"

headers = {
    "accept": "application/json",
    "Authorization": f"{token}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"46de0fe7f964a520944a1fe3","categories":[{"id":13379,"name":"Vietnamese Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/vietnamese_","suffix":".png"}}],"chains":[],"distance":106,"geocodes":{"main":{"latitude":37.795566,"longitude":-122.402009},"roof":{"latitude":37.795566,"longitude":-122.402009}},"link":"/v3/places/46de0fe7f964a520944a1fe3","location":{"address":"505 Washington St","census_block":"060750611012014","country":"US","cross_street":"btw Hotaling St & Sansome St","dma":"San Francisco-Oakland-San Jose","formatted_address":"505 Washington St (btw Hotaling St & Sansome St), San Francisco, CA 94111","locality":"San Francisco","postcode":"94111","region":"CA"},"name":"Sai's Restaurant","related_places":{},"timezone":"America/Los_Angeles"},{"fsq_id":"42829c80f964a5201f221fe3","categories":[{"id":13016,"name":"Lounge","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightlife/default_","suffix":".png"}},{"id":13068,"name":"A

In [36]:
vegan_list = []
for i in response.json()["results"]:
    vegan_list.append(name_coordinates(i))
vegan_list

[{'name': "Sai's Restaurant", 'lat': 37.795566, 'lon': -122.402009},
 {'name': 'Kells Irish Restaurant & Pub',
  'lat': 37.796482,
  'lon': -122.404142},
 {'name': 'Tokyo Express Restaurant', 'lat': 37.793853, 'lon': -122.403915}]

In [37]:
vegan = pd.DataFrame(vegan_list)


### Creating df

In [38]:
# Vegan

vegan['type'] = vegan['type'] = 'vegan'
vegan.drop([1], inplace=True)

# Pub

pub['type'] = pub['type'] = 'pub'
pub.drop([0], inplace=True)
pub.drop([2], inplace=True)

# kinder

close_kinder['type'] = close_kinder['type'] = 'kindergarden'
close_kinder.drop([0,2], inplace=True)

# Starbucks

close_5_starbucks['type'] = close_5_starbucks['type'] = 'coffe'

new_df = pd.concat([close_kinder, vegan], axis=0)
df_def = pd.concat([new_df,pub],axis=0)
df_2 = pd.concat([df_def,close_5_starbucks],axis=0, ignore_index=True)

design_df = close_companies[close_companies['name'] == '99designs']
design_df = design_df.rename(columns={'latitude': 'lat', 'longitude': 'lon'})  # old method
design_df['type'] = design_df['type'] = 'design_company' 
design_df.drop(['total_money_raised','state_code','geometry','Distance_meters'],axis=1,inplace=True)

df_3 = pd.concat([df_2,design_df],axis=0, ignore_index=True)

webly_df = close_companies[close_companies['name'] == 'BrightRoll']
webly_df = webly_df.rename(columns={'latitude': 'lat', 'longitude': 'lon'})  # old method
webly_df['type'] = webly_df['type'] = 'new_venue' 
webly_df.drop(['total_money_raised','state_code','geometry','Distance_meters'],axis=1,inplace=True)

df_vf = pd.concat([df_3,webly_df],axis=0, ignore_index=True)

## FINAL MAP! (We finally have a new venue)

In [65]:
us_lat = 37.795531
us_lon = -122.400598

def_map = Map(location=[us_lat, us_lon], zoom_start = 17)

for index, row in df_vf.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["type"] == "new_venue":        
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="briefcase",
            icon_color = "white",
            radius=45
        )
    elif row["type"] == "pub":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="beer",
            icon_color = "yellow"
        )
    elif row["type"] == "coffe":
        icon = Icon (
            color="darkgreen",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "white"
        )
    elif row["type"] == "kindergarden":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="school",
            icon_color = "white"
        )
    
    elif row["type"] == "design_company":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="pen",
            icon_color = "white"
            
        )
    
    else:
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="leaf",
            icon_color = "white",
            icon_size=(35, 35)
        )
    #3. Marker
    new_marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker.add_to(def_map)

def_map